# Generate release notes

This notebook guides you through the process of creating release notes. Using `generate_release_objects.py` and the OpenAI API, we are able to automate the release notes authoring process. 

This notebook grabs all the release notes information from GitHub when you provide a release URL. It then does some processing to sort by labels and then put everything together. Then, we put it through OpenAI to do some editing. It still needs human editing, which you can do after you run this notebook.

After running the notebook, you'll see new generated release notes added to our Quarto docs site that you can preview and edit further. It will be under `~/site/releases`.

Please have your release URLs ready to use this notebook. You will paste them into the prompt once you run it.

## Contents<a id='toc0_'></a>    
- [Prerequisites](#toc1_)    
- [Setup](#toc2_)    
  - [Import necessary libraries](#toc2_1_)    
  - [Set up OpenAI API](#toc2_2_)    
  - [Set labels](#toc2_3_)    
  - [Collect GitHub URLs](#toc2_4_)    
  - [Set the release date](#toc2_5_)    
- [Extract PR information](#toc3_)    
  - [Create release folder](#toc3_1_)    
  - [Start writing to release notes file](#toc3_2_)    
  - [Set up release notes components](#toc3_3_)    
  - [Set the repository and tag name](#toc3_4_)    
  - [Extract PRs from each URL](#toc3_5_)    
  - [Load PR data](#toc3_6_)    
- [Edit release notes](#toc4_)    
  - [Edit the release notes body](#toc4_1_)    
  - [Load Git diff - DELETE!!!](#toc4_2_)    
  - [Use OpenAI API to interpret the Git Diff - DELETE!!!](#toc4_3_)    
  - [Compare outputs - DELETE!!!!](#toc4_4_)    
  - [Edit each title](#toc4_5_)    
  - [Set labels for each PR](#toc4_6_)    
  - [Assign PR details to PR](#toc4_7_)    
  - [Combine all PR data into the same release notes components](#toc4_8_)    
- [Add release notes to docsite and preview](#toc5_)    
  - [Write release notes to file](#toc5_1_)    
  - [Update sidebar](#toc5_2_)    
  - [Show files to commit](#toc5_3_)    
  - [Preview and edit changes](#toc5_4_)    
- [Next steps](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>Prerequisites [](#toc0_)
You should be on a separate branch associated to the story for the release notes. See [our release notes guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) for the steps needed before running this notebook.

## <a id='toc2_'></a>Setup [](#toc0_)

### <a id='toc2_1_'></a>Import necessary libraries [](#toc0_)

This cell imports any dependencies and some functions from `generate_release_objects.py`.

In [1]:
import requests
import subprocess
import json
import re
import shutil
import numpy as np
import datetime
import openai
from dotenv import load_dotenv
import os

from generate_release_objects import ReleaseURL, PR
from generate_release_objects import get_release_date, write_prs_to_file, collect_github_urls

### <a id='toc2_2_'></a>Set up OpenAI API [](#toc0_)

Running this cell grabs your OpenAI API secret key from your `.env` file. If the relative path to your `.env` file is not `../.env`, change it to your relative path.

In [2]:
def setup_openai_api():
    """Loads .env file and updates the OpenAI API key. 
    
    Replace '../.env' with the relative path to your .env file.

    Modifies:
        openai.api_key
    """
    # Load environment variables
    load_dotenv('../.env') # replace to match your correct path

    # Get the OpenAI API key
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise EnvironmentError("OpenAI API key is not set in .env file.")

    # Set the API key for the OpenAI library
    openai.api_key = api_key

setup_openai_api()

### <a id='toc2_3_'></a>Set labels [](#toc0_)

This cell creates the main sections of the release notes. `label_hierarchy` shows the order in which updates will be shown.

In [3]:

label_to_category = {
    "highlight": "## Release highlights",
    "enhancement": "## Enhancements",
    "deprecation": "## Deprecations",
    "bug": "## Bug fixes",
    "documentation": "## Documentation"
}

categories = { 
    "highlight": [],
    "enhancement": [],
    "deprecation": [],
    "bug": [],
    "documentation": []
}

label_hierarchy = ["highlight", "deprecation", "bug", "enhancement", "documentation"]

### <a id='toc2_4_'></a>Collect GitHub URLs [](#toc0_)

Running this cell will prompt you to enter your GitHub release URLs. Keep pasting them in until you're done, then press enter again.

Example release URL: https://github.com/validmind/documentation/releases/tag/v2.4.4

In [4]:

github_urls = collect_github_urls() # the only big global variable

https://github.com/validmind/frontend/releases/tag/v1.27.17 added.

https://github.com/validmind/frontend/releases/tag/v1.27.17 added.

https://github.com/validmind/documentation/releases/tag/v2.6.7 added.

https://github.com/validmind/documentation/releases/tag/v2.5.25 added.

https://github.com/validmind/validmind-library/releases/tag/v2.6.7 added.



### <a id='toc2_5_'></a>Set the release date [](#toc0_)
Running this cell will prompt you to enter the desired release date. 
The default is 3 business days from today if you leave the prompt empty.

In [5]:

release_datetime = get_release_date()
formatted_release_date = release_datetime.strftime("%Y-%b-%d").lower()
original_release_date = release_datetime.strftime("%B %-d, %Y")

Release date: 2025-01-01 00:00:00



## <a id='toc3_'></a>Extract PR information [](#toc0_)

### <a id='toc3_1_'></a>Create release folder [](#toc0_)

These lines will create a folder inside of `~/site/releases` for the release notes. The folder name is the release date, as per our convention.

In [6]:

directory_path = f"../site/releases/{formatted_release_date}/"
os.makedirs(directory_path, exist_ok=True)
output_file = f"{directory_path}release-notes.qmd"
print(f"release-notes.qmd in {directory_path} created.")

release-notes.qmd in ../site/releases/2025-jan-01/ created.


### <a id='toc3_2_'></a>Start writing to release notes file [](#toc0_)
This block writes the title of the release notes into the final release notes file.

In [7]:

print("Generating & editing release notes ...")

with open(output_file, "w") as file:
    file.write(f"---\ntitle: \"{original_release_date}\"\n---\n\n")


Generating & editing release notes ...


### <a id='toc3_3_'></a>Set up release notes components [](#toc0_)
`release_components` will contain all the components of the release notes in the form of a dictionary. Later, we will merge everything together to create the release notes.

In [8]:
release_components = dict()
release_components.update(categories)
print(f"release components so far: {release_components}")

release components so far: {'highlight': [], 'enhancement': [], 'deprecation': [], 'bug': [], 'documentation': []}


### <a id='toc3_4_'></a>Set the repository and tag name [](#toc0_)
This block checks every URL and assigns its repo name, such as `documentation` or `backend`, and its tag name.

In [9]:
for url in github_urls:
    url.set_repo_and_tag_name()

URL: https://github.com/validmind/frontend/releases/tag/v1.27.17
 Repo name: validmind/frontend
 Tage name: v1.27.17

URL: https://github.com/validmind/frontend/releases/tag/v1.27.17
 Repo name: validmind/frontend
 Tage name: v1.27.17

URL: https://github.com/validmind/documentation/releases/tag/v2.6.7
 Repo name: validmind/documentation
 Tage name: v2.6.7

URL: https://github.com/validmind/documentation/releases/tag/v2.5.25
 Repo name: validmind/documentation
 Tage name: v2.5.25

URL: https://github.com/validmind/validmind-library/releases/tag/v2.6.7
 Repo name: validmind/validmind-library
 Tage name: v2.6.7



### <a id='toc3_5_'></a>Extract PRs from each URL [](#toc0_)
This block gathers all the pull requests from each release URL and stores them within the URL's object data.

In [10]:
for url in github_urls:
    url.extract_prs() # initializes PR objects into a list for each URL

Extracting PRs from https://github.com/validmind/frontend/releases/tag/v1.27.17...

PR #1085 added.

PR #1086 added.

PR #1087 added.

PR #1088 added.

PR #1089 added.

PR #1090 added.

PR #1091 added.

PR #1083 added.

PR #1093 added.

PR #1094 added.

PR #1079 added.

PR #1097 added.

PR #1098 added.

PR #1103 added.

PR #1104 added.

PR #1105 added.

PR #1080 added.

PR #1099 added.

PR #1095 added.

PR #1108 added.

PR #1110 added.

PR #1109 added.

PR #1111 added.

PR #1112 added.

PR #1113 added.

PR #1114 added.

PR #1118 added.

PR #1100 added.

PR #1101 added.

PR #1081 added.

PR #1076 added.

PR #1107 added.

PR #1106 added.

PR #1092 added.

Extracting PRs from https://github.com/validmind/frontend/releases/tag/v1.27.17...

PR #1085 added.

PR #1086 added.

PR #1087 added.

PR #1088 added.

PR #1089 added.

PR #1090 added.

PR #1091 added.

PR #1083 added.

PR #1093 added.

PR #1094 added.

PR #1079 added.

PR #1097 added.

PR #1098 added.

PR #1103 added.

PR #1104 added.


### <a id='toc3_6_'></a>Load PR data [](#toc0_)

Using the JSON data from the PRs, this block extracts and stores information into each PR's object data.

In [11]:
for url in github_urls:
    url.populate_pr_data()

Extracting data from PR #1085 in validmind/frontend...

Extracting data from PR #1086 in validmind/frontend...

Extracting data from PR #1087 in validmind/frontend...

Extracting data from PR #1088 in validmind/frontend...

Extracting data from PR #1089 in validmind/frontend...

Extracting data from PR #1090 in validmind/frontend...

Extracting data from PR #1091 in validmind/frontend...

Extracting data from PR #1083 in validmind/frontend...

Extracting data from PR #1093 in validmind/frontend...

Extracting data from PR #1094 in validmind/frontend...

Extracting data from PR #1079 in validmind/frontend...

Extracting data from PR #1097 in validmind/frontend...

Extracting data from PR #1098 in validmind/frontend...

Extracting data from PR #1103 in validmind/frontend...

Extracting data from PR #1104 in validmind/frontend...

Extracting data from PR #1105 in validmind/frontend...

Extracting data from PR #1080 in validmind/frontend...

Extracting data from PR #1099 in validmind/front

## <a id='toc4_'></a>Edit release notes [](#toc0_)

### <a id='toc4_1_'></a>Edit the release notes body [](#toc0_)

(20s)
Using the prompt below, this block feeds the body of each PR to ChatGPT for editing, skipping PRs labeled as `internal`. If you find that the output is not quite right, edit the prompt and play around with it.

In [12]:
editing_instructions_body = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Adding PR #{pr.pr_number} from {pr.repo_name} to release notes...\n") 
            if pr.extract_external_release_notes(): pr.edit_text_with_openai(False, editing_instructions_body)


Adding PR #1079 from validmind/frontend to release notes...

Adding PR #1095 from validmind/frontend to release notes...

Adding PR #1100 from validmind/frontend to release notes...

Adding PR #1076 from validmind/frontend to release notes...

Adding PR #1106 from validmind/frontend to release notes...

Adding PR #1092 from validmind/frontend to release notes...

Adding PR #1079 from validmind/frontend to release notes...

Adding PR #1095 from validmind/frontend to release notes...

Adding PR #1100 from validmind/frontend to release notes...

Adding PR #1076 from validmind/frontend to release notes...

Adding PR #1106 from validmind/frontend to release notes...

Adding PR #1092 from validmind/frontend to release notes...

Adding PR #552 from validmind/documentation to release notes...

Adding PR #543 from validmind/documentation to release notes...

Adding PR #544 from validmind/documentation to release notes...

Adding PR #488 from validmind/documentation to release notes...

Adding P

### Try automated GitHub PR summary
Using the new github-actions bot, we can fetch their auto-generated summary. This code block fetches the summary.

In [13]:
summary_instructions = """ 
Please turn this PR Summary into a summary for release notes, according to the following guidelines:
- Use simple and neutral language in the active voice.
- Change from numbered list format to paragraph-style text.
- Address users directly in the second person with "you".
- Use present tense by avoiding the use of "will".
"""

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Fetching github comment from PR #{pr.pr_number} in {pr.repo_name}...\n")
            pr.extract_pr_summary_comment()
            pr.convert_summary_to_release_notes(summary_instructions)

Fetching github comment from PR #1079 in validmind/frontend...

Processing PR Summary #1079 in repo validmind/frontend...

Fetching github comment from PR #1095 in validmind/frontend...

No PR Summary found.
Fetching github comment from PR #1100 in validmind/frontend...

Processing PR Summary #1100 in repo validmind/frontend...

Fetching github comment from PR #1076 in validmind/frontend...

Processing PR Summary #1076 in repo validmind/frontend...

Fetching github comment from PR #1106 in validmind/frontend...

Processing PR Summary #1106 in repo validmind/frontend...

Fetching github comment from PR #1092 in validmind/frontend...

Processing PR Summary #1092 in repo validmind/frontend...

Fetching github comment from PR #1079 in validmind/frontend...

Processing PR Summary #1079 in repo validmind/frontend...

Fetching github comment from PR #1095 in validmind/frontend...

No PR Summary found.
Fetching github comment from PR #1100 in validmind/frontend...

Processing PR Summary #1100 

### <a id='toc4_5_'></a>Edit each title [](#toc0_)
This block does the same as above for the titles of each PR. The output below will show:
- The original PR title
- The title after some algorithmic changes
- The title after ChatGPT edits it

If you find that it's not good after editing with ChatGPT, feel free to edit the prompt below.

In [14]:
editing_instructions_title = """
    Please edit the provided technical content according to the following guidelines:

    - Use simple and neutral language in the active voice.
    - Address users directly in the second person with "you".
    - Use present tense by avoiding the use of "will".
    - Apply sentence-style capitalization to text
    - Always capitalize the first letter of text on each line.
    - Rewrite sentences that are longer than 25 words as multiple sentences.
    - Only split text across multiple lines if the text contains more than three sentences.
    - Avoid handwaving references to "it" or "this" by including the text referred to. 
    - Treat short text of less than ten words without a period at the end as a heading. 
    - Enclose any words joined by underscores in backticks (`) if they aren't already.
    - Remove exclamation marks from text.
    - Remove quotes around non-code words.
    - Remove the text "feat:" from the output
    - Maintain existing punctuation at the end of sentences.
    - Maintain all original hyperlinks for reference.
    - Preserve all comments in the format <!--- COMMENT ---> as they appear in the text.
    """

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            print(f"Editing title for PR #{pr.pr_number} in {pr.repo_name}...\n")
            pr.title = pr.data_json['title']
            pr.clean_title(editing_instructions_title)
            print("\n")

Editing title for PR #1079 in validmind/frontend...

After some trimming: Added the ability to Managed Saved Views

After stripping more: Added the ability to Managed Saved Views

After ChatGPT: Added the ability to manage saved views



Editing title for PR #1095 in validmind/frontend...

After some trimming: Bump cross-spawn

After stripping more: Bump cross-spawn

After ChatGPT: 
Bump cross-spawn



Editing title for PR #1100 in validmind/frontend...

After some trimming: Mnt analytics improvements

After stripping more: Mnt analytics improvements

After ChatGPT: Mnt analytics improvements



Editing title for PR #1076 in validmind/frontend...

After some trimming:  - chore: add UI changes for revision history

After stripping more: - chore: add UI changes for revision history

After ChatGPT: Chore: add UI changes for revision history



Editing title for PR #1106 in validmind/frontend...

After some trimming: Deleting a finding should remove it from a validation report

After strip

### <a id='toc4_6_'></a>Set labels for each PR [](#toc0_)
This block takes the label data from each PR and assigns it to the PR.

In [15]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.labels = [label['name'] for label in pr.data_json['labels']]
            print(f"PR #{pr.pr_number} from {pr.repo_name}: {pr.labels}\n")

PR #1079 from validmind/frontend: ['enhancement']

PR #1095 from validmind/frontend: ['dependencies']

PR #1100 from validmind/frontend: ['enhancement']

PR #1076 from validmind/frontend: ['enhancement']

PR #1106 from validmind/frontend: ['bug']

PR #1092 from validmind/frontend: ['enhancement']

PR #1079 from validmind/frontend: ['enhancement']

PR #1095 from validmind/frontend: ['dependencies']

PR #1100 from validmind/frontend: ['enhancement']

PR #1076 from validmind/frontend: ['enhancement']

PR #1106 from validmind/frontend: ['bug']

PR #1092 from validmind/frontend: ['enhancement']

PR #552 from validmind/documentation: ['documentation']

PR #543 from validmind/documentation: ['documentation']

PR #544 from validmind/documentation: ['documentation']

PR #488 from validmind/documentation: ['documentation']

PR #510 from validmind/documentation: ['documentation']

PR #213 from validmind/validmind-library: []

PR #214 from validmind/validmind-library: ['enhancement']

PR #249 from

### <a id='toc4_7_'></a>Assign PR details to PR [](#toc0_)
This block compiles all the data we found earlier for each PR into one place. 

In [16]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json: 
            pr.pr_details = {
            'pr_number': pr.pr_number,
            'title': pr.cleaned_title,
            'full_title': pr.data_json['title'],
            'url': pr.data_json['url'],
            'labels': ", ".join(pr.labels),
            'notes': pr.edited_text
            }
            print(f"PR #{pr.pr_number} from {pr.repo_name} added.\n")


PR #1079 from validmind/frontend added.

PR #1095 from validmind/frontend added.

PR #1100 from validmind/frontend added.

PR #1076 from validmind/frontend added.

PR #1106 from validmind/frontend added.

PR #1092 from validmind/frontend added.

PR #1079 from validmind/frontend added.

PR #1095 from validmind/frontend added.

PR #1100 from validmind/frontend added.

PR #1076 from validmind/frontend added.

PR #1106 from validmind/frontend added.

PR #1092 from validmind/frontend added.

PR #552 from validmind/documentation added.

PR #543 from validmind/documentation added.

PR #544 from validmind/documentation added.

PR #488 from validmind/documentation added.

PR #510 from validmind/documentation added.

PR #213 from validmind/validmind-library added.

PR #214 from validmind/validmind-library added.

PR #249 from validmind/validmind-library added.

PR #198 from validmind/validmind-library added.

PR #243 from validmind/validmind-library added.

PR #213 from validmind/validmind-libra

### <a id='toc4_8_'></a>Combine all PR data into the same release notes components [](#toc0_)
Now, we can take all the details we compiled above and append them to our final release notes components. Since we want to show features in order of importance, we sort by the priority of the label.

In [17]:

for url in github_urls:
    for pr in url.prs:
        if pr.data_json:
            print(f"Adding PR #{pr.pr_number} from {pr.repo_name}...\n")
            assigned = False 
            for priority_label in label_hierarchy:
                if priority_label in pr.labels:
                    release_components[priority_label].append(pr.pr_details)
                    assigned = True
                    break
            if not assigned:
                release_components.setdefault('other', []).append(pr.pr_details)

Adding PR #1079 from validmind/frontend...

Adding PR #1095 from validmind/frontend...

Adding PR #1100 from validmind/frontend...

Adding PR #1076 from validmind/frontend...

Adding PR #1106 from validmind/frontend...

Adding PR #1092 from validmind/frontend...

Adding PR #1079 from validmind/frontend...

Adding PR #1095 from validmind/frontend...

Adding PR #1100 from validmind/frontend...

Adding PR #1076 from validmind/frontend...

Adding PR #1106 from validmind/frontend...

Adding PR #1092 from validmind/frontend...

Adding PR #552 from validmind/documentation...

Adding PR #543 from validmind/documentation...

Adding PR #544 from validmind/documentation...

Adding PR #488 from validmind/documentation...

Adding PR #510 from validmind/documentation...

Adding PR #213 from validmind/validmind-library...

Adding PR #214 from validmind/validmind-library...

Adding PR #249 from validmind/validmind-library...

Adding PR #198 from validmind/validmind-library...

Adding PR #243 from vali

## <a id='toc5_'></a>Add release notes to docsite and preview [](#toc0_)

### <a id='toc5_1_'></a>Write release notes to file [](#toc0_)
Now that `release_components` contains everything we need for the release notes, we can write it to our release notes file.

In [18]:
# Write categorized PRs to the file
with open(output_file, "a") as file:
    write_prs_to_file(file, release_components, label_to_category)
    print(f"Release notes added to {file.name}.")


Release notes added to ../site/releases/2025-jan-01/release-notes.qmd.


### <a id='toc5_2_'></a>Update sidebar [](#toc0_)
This block will go into our `_quarto.yml` file and add the new release notes so it shows up on the sidebar of the docsite. 

In [19]:

def update_quarto_yaml(release_date):
    """Updates the _quarto.yml file to include the release notes file so it can be accessed on the website.

    Params:
        release_date - release notes use the release date as the file name.
    
    Modifies:
        _quarto.yml file
    """
    yaml_filename = "../site/_quarto.yml"
    temp_yaml_filename = "../site/_quarto_temp.yml"

    # Copy the original YAML file to a temporary file
    shutil.copyfile(yaml_filename, temp_yaml_filename)

    with open(temp_yaml_filename, 'r') as file:
        lines = file.readlines()

    # Format the release date for insertion into the YAML file
    formatted_release_date = release_date.strftime("%Y-%b-%d").lower()

    with open(yaml_filename, 'w') as file:
        add_release_content = False
        insert_index = -1

        for i, line in enumerate(lines):
            file.write(line)
            if line.strip() == "# MAKE-RELEASE-NOTES-EMBED-MARKER":
                add_release_content = True
                insert_index = i

            if add_release_content and i == insert_index:
                file.write(f'        - releases/{formatted_release_date}/release-notes.qmd\n')
                add_release_content = False

    # Remove the temporary file
    os.remove(temp_yaml_filename)
    
    print(f"Added release notes to _quarto.yml, line {insert_index + 2}")

update_quarto_yaml(release_datetime)

Added release notes to _quarto.yml, line 173


### <a id='toc5_3_'></a>Show files to commit [](#toc0_)

In [20]:
# After completing all tasks, print git status to show output files
try:
    result = subprocess.run(["git", "status", "--short"], check=True, text=True, capture_output=True)
    lines = result.stdout.split('\n')
    print("Files to commit:")
    for line in lines:
        if line.startswith((' M', '??', 'A ')):
            print(line)
except subprocess.CalledProcessError as e:
    print("Failed to run git status:", e)

Files to commit:
 M generate-release-notes.ipynb
 M ../site/_quarto.yml
?? __pycache__/generate_release_objects.cpython-310.pyc
?? __pycache__/yearly_cleanup.cpython-310.pyc
?? ../site/releases/2025-jan-01/


### <a id='toc5_4_'></a>Preview and edit changes [](#toc0_)
Run this cell to preview your changes, and make edits to the release notes file you just generated. See our [internal guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) on editing release notes.

In [ ]:
%%bash
cd ../site
quarto preview

Preparing to preview
[1/5] releases/2024-releases.qmd
[2/5] releases/2025-jan-01/release-notes.qmd
[3/5] training/validator-fundamentals/validator-fundamentals.qmd
[4/5] training/developer-fundamentals/developer-fundamentals.qmd
[5/5] index.qmd

make

Updating Python documentation ...
Copying Python documentation into docs site ...

Watching files for changes
Browse at http://localhost:3164/
GET: /

GET: /about/overview.html
GET: /releases/2025-jan-01/release-notes.html


**When you're done with the preview, please restart the kernel.**

## <a id='toc6_'></a>Next steps [](#toc0_)

Now that you've generated, previewed, and edited the release notes, it's time to send a commit and start a PR! Make sure you're on the branch associated to the story for the release notes. Double check with our [internal guide](https://www.notion.so/validmind/On-release-notes-20de4e7ea03f402587514f6c9eda3bb1) to see if you missed anything.